In [2]:
from cookbook.interface import PluginInstanceRedisInterface
import os
import sys

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379
redis_password = ''

# When your PluginService is running, you can get your channel value from the Logs, or from the query parameter in your open browser.
# Update this value to match that, so that your commands will run against your live workspace.
redis_channel = os.environ.get("REDIS_CHANNEL")

# Increase the recursion limit in order to properly serialize Complexes
# recursion_limit = 10000
# sys.setrecursionlimit(recursion_limit)

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
print('Hello')

Hello


In [1]:
from rmsd_v2.RMSDV2 import RMSDV2

class RMSDV2RedisInstance(PluginInstanceRedisInterface):
    pass

NameError: name 'PluginInstanceRedisInterface' is not defined

In [11]:
plugin_instance = RMSDV2()

In [3]:
plugin_instance.request_complex_list()

Sending request_complex_list Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
